<a href="https://colab.research.google.com/github/Anshul4321123/ML-Models/blob/main/MOIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import pandas as pd

In [77]:
alert_df = pd.read_csv("MOIG_Alert.csv")
breach_df = pd.read_csv("MOIG_Threshold_Breach.csv")
threshold_df = pd.read_csv("MOIG_Analytics_threshold.csv")


In [78]:
breach_with_threshold = pd.merge(
    breach_df,
    threshold_df[["THRESHOLD_ID", "THRESHOLD_VALUE", "MAX_VALUE"]],
    on="THRESHOLD_ID",
    how="left"
)

In [79]:
breach_with_threshold["SEVERITY_SCORE"] = (
    ((breach_with_threshold["BREACH_VALUE"] - breach_with_threshold["THRESHOLD_VALUE"])) /
    ((breach_with_threshold["MAX_VALUE"] - breach_with_threshold["THRESHOLD_VALUE"]))
)


In [80]:
severity_per_alert = breach_with_threshold.groupby("ALERT_ID")["SEVERITY_SCORE"].max().reset_index()


In [81]:
final_alerts = pd.merge(alert_df, severity_per_alert, on="ALERT_ID", how="left")

In [82]:
final_alerts.to_csv("MOIG_Alerts_with_Severity.csv", index=False)

In [83]:
print(final_alerts[["ALERT_ID", "ALERT_SCORE", "ALERT_SEVERITY", "SEVERITY_SCORE"]])

     ALERT_ID  ALERT_SCORE ALERT_SEVERITY  SEVERITY_SCORE
0   MOIG_5351          0.9           High        708543.0
1   MOIG_5352          0.9           High        708543.0
2   MOIG_5353          0.9           High        708543.0
3   MOIG_5354          0.9           High        708543.0
4   MOIG_5355          0.9           High        147611.0
5   MOIG_5351          0.9           High        708543.0
6   MOIG_5351          0.9           High        708543.0
7   MOIG_5352          0.9           High        708543.0
8   MOIG_5351          0.9           High        708543.0
9   MOIG_5352          0.9           High        708543.0
10  MOIG_5353          0.9           High        708543.0
11  MOIG_5351          0.9           High        708543.0
12  MOIG_5352          0.9           High        708543.0
13  MOIG_5353          0.9           High        708543.0
14  MOIG_5354          0.9           High        708543.0
15  MOIG_5355          0.9           High        147611.0


In [84]:
import pandas as pd

alert_df = pd.read_csv("MOIG_Alert.csv")
breach_df = pd.read_csv("MOIG_Threshold_Breach.csv")
threshold_df = pd.read_csv("MOIG_Analytics_threshold.csv")

merged = pd.merge(
    breach_df,
    threshold_df[["THRESHOLD_ID", "THRESHOLD_VALUE", "MAX_VALUE"]],
    on="THRESHOLD_ID",
    how="left"
)

merged["RAW_SEVERITY"] = (
    (merged["BREACH_VALUE"] - merged["THRESHOLD_VALUE"]) /
    (merged["MAX_VALUE"] - merged["THRESHOLD_VALUE"])
)

merged = merged.replace([float('inf'), -float('inf')], float('nan')).dropna(subset=["RAW_SEVERITY"])

min_val = merged["RAW_SEVERITY"].min()
max_val = merged["RAW_SEVERITY"].max()

if max_val > min_val:
    merged["SEVERITY_SCORE"] = (merged["RAW_SEVERITY"] - min_val) / (max_val - min_val)
else:
    merged["SEVERITY_SCORE"] = 0.0  # fallback if all values are the same

severity_scores = merged.groupby("ALERT_ID")["SEVERITY_SCORE"].max().reset_index()

final_alerts = pd.merge(alert_df, severity_scores, on="ALERT_ID", how="left")
final_alerts["SEVERITY_SCORE"] = final_alerts["SEVERITY_SCORE"].fillna(0)

final_alerts.to_csv("MOIG_Alerts_with_Normalized_Severity.csv", index=False)
print(final_alerts[["ALERT_ID", "ALERT_SCORE", "ALERT_SEVERITY", "SEVERITY_SCORE"]])

     ALERT_ID  ALERT_SCORE ALERT_SEVERITY  SEVERITY_SCORE
0   MOIG_5351          0.9           High        1.000000
1   MOIG_5352          0.9           High        1.000000
2   MOIG_5353          0.9           High        1.000000
3   MOIG_5354          0.9           High        1.000000
4   MOIG_5355          0.9           High        0.208332
5   MOIG_5351          0.9           High        1.000000
6   MOIG_5351          0.9           High        1.000000
7   MOIG_5352          0.9           High        1.000000
8   MOIG_5351          0.9           High        1.000000
9   MOIG_5352          0.9           High        1.000000
10  MOIG_5353          0.9           High        1.000000
11  MOIG_5351          0.9           High        1.000000
12  MOIG_5352          0.9           High        1.000000
13  MOIG_5353          0.9           High        1.000000
14  MOIG_5354          0.9           High        1.000000
15  MOIG_5355          0.9           High        0.208332


In [85]:
import pandas as pd

analytics_df = pd.read_csv("MOIG_Analytics_threshold.csv")
breach_df = pd.read_csv("MOIG_Threshold_Breach.csv")

keywords = ['ignition', 'momentum']
filtered_thresholds = analytics_df[
    analytics_df['THRESHOLD_NAME'].str.lower().str.contains('|'.join(keywords)) |
    analytics_df['THRESHOLD_DESCRIPTION'].str.lower().str.contains('|'.join(keywords))
]

merged_df = pd.merge(
    breach_df,
    filtered_thresholds[['THRESHOLD_ID', 'THRESHOLD_NAME', 'THRESHOLD_DESCRIPTION', 'THRESHOLD_VALUE', 'MAX_VALUE']],
    on='THRESHOLD_ID',
    how='inner'
)

def calculate_severity(row):
    try:
        breach = row['BREACH_VALUE']
        threshold = row['THRESHOLD_VALUE']
        max_val = row['MAX_VALUE']
        if pd.isna(threshold) or pd.isna(max_val) or max_val == threshold:
            return 0.0
        return min(1.0, max(0.0, (breach - threshold) / (max_val - threshold)))
    except:
        return 0.0

merged_df['SEVERITY_SCORE'] = merged_df.apply(calculate_severity, axis=1)

merged_df = merged_df.sort_values(by='SEVERITY_SCORE')

merged_df.to_csv("Ignition_Breaches_With_Severity.csv", index=False)
print(merged_df[["ALERT_ID", "SEVERITY_SCORE"]].head())


     ALERT_ID  SEVERITY_SCORE
63  MOIG_5355             0.0
62  MOIG_5354             0.0
61  MOIG_5355             0.0
60  MOIG_5353             0.0
59  MOIG_5354             0.0


In [86]:
import pandas as pd

alert_df = pd.read_csv("MOIG_Alert.csv")
breach_df = pd.read_csv("MOIG_Threshold_Breach.csv")
threshold_df = pd.read_csv("MOIG_Analytics_threshold.csv")

breach_with_txn = pd.merge(
    breach_df,
    alert_df[["ALERT_ID", "TRIGGERING_TRANSACTION_INTERNAL_ID"]],
    on="ALERT_ID",
    how="left"
)

merged_all = pd.merge(
    breach_with_txn,
    threshold_df[["THRESHOLD_ID", "THRESHOLD_VALUE", "MAX_VALUE"]],
    on="THRESHOLD_ID",
    how="left"
)

merged_all["NORMALIZED_BREACH"] = (
    (merged_all["BREACH_VALUE"] - merged_all["THRESHOLD_VALUE"]) /
    (merged_all["MAX_VALUE"] - merged_all["THRESHOLD_VALUE"])
)

merged_all = merged_all.replace([float('inf'), -float('inf')], float('nan')).dropna(subset=["NORMALIZED_BREACH"])
merged_all = merged_all[merged_all["MAX_VALUE"] > merged_all["THRESHOLD_VALUE"]]

positive_only = merged_all[merged_all["NORMALIZED_BREACH"] > 0]
max_magnitude = positive_only["NORMALIZED_BREACH"].quantile(0.99)

merged_all["MAPPED_SCORE"] = merged_all["NORMALIZED_BREACH"].clip(lower=0, upper=max_magnitude)
merged_all["MAPPED_SCORE"] = merged_all["MAPPED_SCORE"] / max_magnitude
merged_all.loc[merged_all["NORMALIZED_BREACH"] < 0, "MAPPED_SCORE"] = 0

trader_scores = merged_all.groupby("TRIGGERING_TRANSACTION_INTERNAL_ID")["MAPPED_SCORE"].mean().reset_index()
trader_scores = trader_scores.rename(columns={"MAPPED_SCORE": "TRADER_SEVERITY_SCORE"})

print(trader_scores.sort_values("TRADER_SEVERITY_SCORE", ascending=False).head(10))
trader_scores.to_csv("Trader_Severity_Scores.csv", index=False)


   TRIGGERING_TRANSACTION_INTERNAL_ID  TRADER_SEVERITY_SCORE
3                 trx_2025-05-07_2497               0.078671
14               trx_2025-05-07_94699               0.078671
7                trx_2025-05-07_67057               0.076075
1                 trx_2025-05-07_2439               0.076075
9                trx_2025-05-07_82867               0.076075
12               trx_2025-05-07_94651               0.076075
10               trx_2025-05-07_82879               0.062585
2                 trx_2025-05-07_2462               0.062585
13               trx_2025-05-07_94692               0.062585
6                trx_2025-05-07_63446               0.054788


In [87]:
import pandas as pd

threshold_df = pd.read_csv("MOIG_Analytics_threshold.csv")
breach_df = pd.read_csv("MOIG_Threshold_Breach.csv")

merged = pd.merge(
    breach_df,
    threshold_df[["THRESHOLD_ID", "THRESHOLD_VALUE", "MAX_VALUE"]],
    on="THRESHOLD_ID",
    how="left"
)

merged["NORMALIZED_BREACH"] = (
    (merged["BREACH_VALUE"] - merged["THRESHOLD_VALUE"]) /
    (merged["MAX_VALUE"] - merged["THRESHOLD_VALUE"])
)

merged = merged.replace([float('inf'), -float('inf')], float('nan'))
merged = merged.dropna(subset=["NORMALIZED_BREACH"])
merged = merged[merged["MAX_VALUE"] > merged["THRESHOLD_VALUE"]]

positive_only = merged[merged["NORMALIZED_BREACH"] > 0]
max_magnitude = positive_only["NORMALIZED_BREACH"].quantile(0.99)

merged["MAPPED_SCORE"] = merged["NORMALIZED_BREACH"].clip(lower=0, upper=max_magnitude)
merged["MAPPED_SCORE"] = merged["MAPPED_SCORE"] / max_magnitude
merged.loc[merged["NORMALIZED_BREACH"] < 0, "MAPPED_SCORE"] = 0  # Handle negatives

alert_severity = merged.groupby("ALERT_ID")["MAPPED_SCORE"].mean().reset_index()
alert_severity = alert_severity.rename(columns={"MAPPED_SCORE": "ALERT_SEVERITY_SCORE"})

alert_severity.to_csv("Alert_Severity_Scores.csv", index=False)

print(alert_severity.sort_values("ALERT_SEVERITY_SCORE", ascending=False).head(10))


    ALERT_ID  ALERT_SEVERITY_SCORE
3  MOIG_5354              0.078671
1  MOIG_5352              0.076075
2  MOIG_5353              0.062585
0  MOIG_5351              0.054788
4  MOIG_5355              0.022122
